In [1]:
import numpy as np
import random
from PIL import Image, ImageDraw
import pygame
import sys
import time
version = "Version 0.9.0"

#spike net to make the decisions
#evolutionary algorithm for the next generation
#maybe just skip both and create a network with no hiddens where its weights are decided by the genes
#you can still add heuristics

#GUI: select pop, pause, fast forward etc
#performance
#minimap?
#separate logic and drawing, save logic as demo and play it
#make "elevation" a spectrum, from -3 to 3, minus values get water, plus values get elevation

#make a map drawer +
#isometric view ++
#pop decisions ++
#entity interact ++
#handle entities  +++
#add a border tile + 

pygame 2.1.2 (SDL 2.0.16, Python 3.10.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def generate_map(grid,rivers):
    x = grid[0]+70
    y = grid[1]+70
    border = np.full((x,y),3)
    map_full = np.zeros((x-70,y-70))
    #map_full = np.zeros((x,y)) #grass
    #map_full = np.full((x-2,y-2),6) #desert
    
    for river in range(rivers):
        riverwidth = random.randint(5,8)
        rando = random.randint(120+riverwidth*2,grid[1]-(120+riverwidth*2))        
        lzm = [[0,0]]

        for l in range(grid[0]-2):
            lzm.append([0,random.randint(-1,1)])
            
            if random.random()+lzm[l][0]<0.45:
                if random.random()<0.8:
                    cld = random.randint(-1,1)
                else: cld = lzm[l][1]
                
                rando+= cld
                lzm[l][0] = 0.2
                lzm[l][1] = cld
                
                for x in range(0,5):
                    if random.random()+lzm[l][0]<(0.4+(x/10)):
                        lzm[l][0]=(0.4+(x/10))
                        
            for w in range(riverwidth):
                map_full[rando+w][l] = 2  
                cvmhj = [-1,1]
                cvk = [2,3]
                for ww in range(2):
                    if map_full[rando+w+cvmhj[ww]][l]!=cvk[ww]:
                        for q in range((riverwidth**2)):
                            www=map_full[rando+w+(q*cvmhj[ww])][l]
                            if www==0 or www==6:
                                map_full[rando+w+(q*cvmhj[ww])][l] = 1

    border[35:-35,35:-35] = map_full
    map_full = border
    return np.array(map_full)

In [3]:
def generate_population(map_full,populations,exclude,entity,pattern,entitymap=[]):
    length=len(map_full[0])-1
    
    if len(entitymap)==0:
        if entity==1:
            entity_full = np.zeros(map_full.shape,dtype=bool)    
        else:entity_full = np.zeros(map_full.shape)
    else:
        entity_full = entitymap
    
    if pattern =="random":
        for population in range(populations):
            random_y = random.randint(0,length)
            random_x = random.randint(0,length)
            targettile = int(map_full[random_y][random_x])
            if targettile not in exclude:
                    entity_full[random_y][random_x] = entity
    
    elif pattern == "patches":
        dimension = int(np.sqrt(populations))
        ent_array = np.full((dimension,dimension),entity)
        
    return entity_full

In [4]:
class Person:
    def __init__(self,placement,energy=None):
        self.placement = placement
        self.index = len(poplist)
        self.moves_in_this_direction = 0
        self.same_movement = False

        if energy==None:
            self.energy = 10000
        else:
            self.energy=energy

        self.movement_dict = [(0,1),(0,-1),(1,0),(-1,0)]
        self.mv = self.movement_dict[0]
        get_pop_placement[(placement[0],placement[1])] = self
        poplist.append(self)

    def line_of_sight(self,map_full,entity_full):
        sight_array =map_full[self.placement[1]-4:self.placement[1]+5,self.placement[0]-4:self.placement[0]+5]
        entity_sight_array =entity_full[self.placement[1]-4:self.placement[1]+5,self.placement[0]-4:self.placement[0]+5]
        
        banans = np.where(entity_sight_array==3)
        closest_banan = [0,0,999]
        for indice in range(len(banans[0])):
            banan_relative_x = (4-banans[1][indice])
            banan_relative_y = (4-banans[0][indice])
            placement = np.array((0,0))
            bananplacement = np.array((banan_relative_y,banan_relative_x))
            distance = np.linalg.norm((placement-bananplacement))

            if distance<closest_banan[2]:
                closest_banan[0] = banan_relative_x
                closest_banan[1] = banan_relative_y
                closest_banan[2] = distance
        
        return closest_banan
        #if closest_banan[2]==1:
        #    self.interact((closest_banan[0],closest_banan[1]))

        
    def move(self,movement):
        #make it so that you get a np.where of the (x,x) around the pop
        #this would make it easier to move directionally
        #e.g "banan in 2 tiles left" therefore "move left" but not explicitly west, just "move in the direction of the banan"
        #keep the place of birth as (x,y) as well
        #maybe even give characters a chance to "save" certain locations as (x,y)

        revert = [self.placement[0],self.placement[1]]        
        point_x = self.placement[0]-movement[0]
        point_y = self.placement[1]-movement[1]
        self.placement = [point_x,point_y]

        point_on_map = int(map_full[self.placement[1]][self.placement[0]])
        point_on_entity = int(entity_full[self.placement[1]][self.placement[0]])

        if (point_on_map in [2,3] or point_on_entity!=0):
            self.placement = revert
        
        if movement == self.mv:
            self.same_movement = True
            self.moves_in_this_direction+=1
        else:
            self.moves_in_this_direction=0
            self.same_movement = False
            self.energy-=10
        self.mv = movement
    
    def interact(self,direction):
        point_x = self.placement[0]-direction[0]
        point_y = self.placement[1]-direction[1]
        
        print(point_x,point_y)

        point_on_entity = int(entity_full[point_y][point_x])
        point_on_map = int(map_full[point_y][point_x])

        if point_on_entity in interactable:
            entity_full[point_y][point_x]=interact_dictionary.get(point_on_entity)
            if point_on_entity in [1,2]:
                self.energy-=100
            elif point_on_entity==3:
                self.energy+=2000

In [5]:
def generate_population(map_full,populations,exclude,entity,pattern,entitymap=[]):
    length=len(map_full[0])-1
    
    if len(entitymap)==0:
        if entity==1:
            entity_full = np.zeros(map_full.shape)    
        else:entity_full = np.zeros(map_full.shape)
    else:
        entity_full = entitymap
    
    if pattern =="random":
        for population in range(populations):
            random_y = random.randint(0,length)
            random_x = random.randint(0,length)
            targettile = int(map_full[random_y][random_x])
            if targettile not in exclude:
                    entity_full[random_y][random_x] = entity
    
    elif pattern == "patches":
        dimension = int(np.sqrt(populations))
        ent_array = np.full((dimension,dimension),entity)
        
    return entity_full

In [6]:
def load_images(name_list,k,directory=None):

    image_names_dict = {}
    image_map_dict = {}
    image_number = 0
    for index,image_name in enumerate(name_list):
        image_names_dict[index+k] = image_name
        img = pygame.image.load(f"{image_name}.png").convert_alpha()
        image_map_dict[index+k] = img
        image_number+=1

    return name_list, image_names_dict,image_map_dict,image_number

In [7]:
def draw_map(map_array,entity_array):
    map_dict = {0:(120,180,60),1:(105,119,46),2:(157,206,243),3:(0,0,0)}
    entity_dict = {2:(67,46,11),1:(110,110,110),3:(242,230,70)}
    img = Image.new("RGB", (map_array.shape))
    img1 = ImageDraw.Draw(img)  
    img1.rectangle( [(0,0),(map_array.shape)] ,fill="white",outline="white")
    
    for element in map_dict:
        b,a = np.where(map_array==element)
        color = map_dict.get(element)
        for index,x in enumerate(a):
            y = b[index]
            img.putpixel((x,y),(color))
    
    for element in entity_dict:
        b,a = np.where(entity_array==element)
        color = entity_dict.get(element)
        for index,x in enumerate(a):
            if x!=0:
                y = b[index]
                img.putpixel((x,y),(color))
    nym = ""
    for x in range(11):
        nym = nym+str(random.randint(0,10))
    img.save(f"maps/{nym}.png")
    img.show()

In [8]:
visible = True

if visible==True:
    pygame.init()
    
    screen = pygame.display.set_mode((1500, 864))    #set the display mode, window title and FPS clock
    pygame.display.set_caption('screen')
    FPSCLOCK = pygame.time.Clock()

    my_font = pygame.font.Font('manaspc.ttf', 11)
    font2 = pygame.font.Font('manaspc.ttf', 20)
    sickfont = pygame.font.Font("manaspc.ttf",10)

    tile_list, tile_names, tile_map, tiles_number = load_images(["iso_grass","iso_fertile","iso_water","border"],0)
    entity_list, entity_names, entity_map, entity_number = load_images(["iso_rocks","iso_tree","iso_banan","branches","broken rocks"],1)

    pop1 = pygame.image.load("pop1.png").convert_alpha()
    pop2 = pygame.image.load("pop2.png").convert_alpha()
    pop3 = pygame.image.load("pop3.png").convert_alpha()
    pop4 = pygame.image.load("pop4.png").convert_alpha()
    pop5 = pygame.image.load("pop5.png").convert_alpha()
    pop6 = pygame.image.load("pop6.png").convert_alpha()
    pop7 = pygame.image.load("pop7.png").convert_alpha()
    pop8 = pygame.image.load("pop8.png").convert_alpha()
    pop_movement_dict = {(-1,-1):pop1,(1,1):pop2,(-1,1):pop3,(1,-1):pop4,(0,1):pop5,(0,-1):pop6,(1,0):pop7,(-1,0):pop8}
    characters_movements_dicts = [pop_movement_dict]
    pop_map = {1:pop_movement_dict.get((-1,-1))}

    gui_bottom = pygame.image.load("gui_bottom.png").convert_alpha()

    speed = 1

    display_tiles_x = 65
    display_tiles_y = 65

    TILEWIDTH = 64 
    TILEHEIGHT = 64
    TILEHEIGHT_HALF = TILEHEIGHT /2
    TILEWIDTH_HALF = TILEWIDTH /2
    editor_mode = True

    list_of_dictionaries = [tile_map,entity_map,pop_map]

    CURRENT_STATE = 0
    switchu = {1:0,0:1}
    btn_values = [[50,114],[50,562]]
    guiwidth = [32,512]

    SLC = ("tile",1)


In [9]:
poplist = []
get_pop_placement = {}
interact_dictionary = {1:5,2:4,3:0}
interactable = [1,2,3]

map_full = generate_map(grid=(500,500),rivers=1)
map_visible = map_full[1:11]

entity_full = generate_population(map_full,100,[2,3],1,"random") #rock1
entity_full = generate_population(map_full,200,[2,3],2,"random",entity_full)  #tree
entity_full = generate_population(map_full,12000,[2,3],3,"random",entity_full)  #banan
entity_visible = entity_full[1:11]

time = 0
population = 1
for x in range(population):
    x = random.randint(40,500)
    y = random.randint(40,500)
    if (map_full[y][x] not in [2] and entity_full[y][x]==0):
        person = Person((x,y))

camera_x = poplist[0].placement[0]+65-25
camera_y = poplist[0].placement[1]+65-25

print(camera_x,camera_y)

draw_map(map_full,entity_full)
movements_possible_list = [(0,1),(0,-1),(1,0),(-1,0)]
grid = (map_full.shape[0],map_full.shape[1])

412 166


In [10]:
while True:
    time +=1
    keys = pygame.key.get_pressed()

    for event in pygame.event.get(): #to quit
        if event.type == pygame.QUIT:
            pygame.display.quit()
            pygame.quit()
            sys.exit()            
    
    horas = int(time/600)
    mins =int((time%600)/10)
    days = int(horas/24)
    day_hours = int(horas%24)
    
    get_pop_placement = {}
    pop_full = np.zeros(map_full.shape)
    for index,popl in enumerate(poplist):
        if popl.energy<0:
                poplist.remove(popl)

        get_pop_placement[(popl.placement[0],popl.placement[1])] = popl
        pop_full[popl.placement[1],popl.placement[0]] = 1
        revert = [camera_y,camera_x]
        
        #if random.random()<0.1:
        #    popl.line_of_sight(map_full,entity_full)
        popl.energy-=1

    if visible==True:
        if camera_x<70:
            camera_x = 70
        elif camera_x>grid[0]:
            camera_x = grid[0]
        if camera_y<70:
            camera_y = 70
        elif camera_y>grid[0]:
            camera_y = grid[0]
            
        screen.fill((0,0,0))
        gui_buttons_map = {}
        
        if (keys[pygame.K_RIGHT] and keys[pygame.K_UP]):
            camera_x -=speed
        elif (keys[pygame.K_LEFT] and keys[pygame.K_UP]):
            camera_y -=speed
        elif (keys[pygame.K_LEFT] and keys[pygame.K_DOWN]):
            camera_x +=speed
        elif (keys[pygame.K_RIGHT] and keys[pygame.K_DOWN]):
            camera_y +=speed        
        elif (keys[pygame.K_RIGHT]):
            camera_x -=speed
            camera_y +=speed
        elif (keys[pygame.K_LEFT]):
            camera_x +=speed
            camera_y -=speed
        elif (keys[pygame.K_DOWN]):
            camera_x+=speed
            camera_y+=speed
        elif (keys[pygame.K_UP]):
            camera_y-=speed
            camera_x-=speed
        


        map_visible = map_full[camera_y-display_tiles_y:camera_y,camera_x-display_tiles_x:camera_x]
        entity_visible = entity_full[camera_y-display_tiles_y:camera_y,camera_x-display_tiles_x:camera_x]
        pop_visible = pop_full[camera_y-display_tiles_y:camera_y,camera_x-display_tiles_x:camera_x]
        list_of_visible_arrays = [map_visible,entity_visible,pop_visible]

        for y, row in enumerate(map_visible):
            for x, tile in enumerate(row):
                cartesian_x = y * TILEWIDTH_HALF
                cartesian_y = x * TILEHEIGHT_HALF  
                isometric_x = (cartesian_x - cartesian_y) 
                isometric_y = (cartesian_x + cartesian_y)/2
                centered_x = screen.get_rect().centerx + isometric_x
                centered_y = screen.get_rect().centery/2 + isometric_y

                for index_of_list, list_itself in enumerate(list_of_visible_arrays): 
                    tile = list_itself[y][x]
                    tile = (int(tile))           
                    
                    if (index_of_list == 0) or (index_of_list == 1 and tile!=0): #tiles
                        image = list_of_dictionaries[index_of_list].get(tile)

                    elif (index_of_list==2 and tile!=0): #pop
                        point_x = x+camera_x-display_tiles_x
                        point_y = y+camera_y-display_tiles_y

                        character = get_pop_placement.get((point_x,point_y))
                        image = characters_movements_dicts[tile-1].get(character.mv)

                    screen.blit(image, (centered_x, centered_y-800))
    
        for ririri in range(tiles_number+entity_number+4):
            screen.blit(gui_bottom, (0+(ririri*64),800))    
        for tile1 in range(tiles_number):
            gui_buttons_map[tile1] = ("tile",tile1)
            screen.blit(tile_map.get(tile1),((tile1*64),800))
        tile1+=1
        gui_buttons_map[tile1] = ("tile",0)
        for entity1 in range(entity_number):
            entity1=entity1+1
            gui_buttons_map[tile1+entity1] = ("entity",entity1)
            screen.blit(entity_map.get(entity1),((tile1+entity1)*64,800))
        gui_buttons_map[tile1+entity1+1] = ("entity",0)


        if pygame.mouse.get_pressed()[0]:
            clicked_x,clicked_y = pygame.mouse.get_pos()
            clicked_x=clicked_x
            clicked_y=clicked_y+800
            text_surface2 = font2.render((f"Coordinates on screen (x,y): {clicked_x},{clicked_y-800}, camera (x,y): {camera_x},{camera_y}"), False, (33, 33, 33))
            screen.blit(text_surface2,(0,25))
            
            if (900>clicked_y-800>800)==False: #if click is not inside the bottom gui borders
                isometric_corrected_x = int(((2*clicked_y - clicked_x)*0.5)/32)+4
                isometric_corrected_y = int(((((2*clicked_y + clicked_x)*0.5)/32))-20)

                e_text=""
                p_text=""
                    
                if entity_visible[isometric_corrected_y][isometric_corrected_x]!=0:
                    e_text= f" It has a {entity_names.get(entity_visible[isometric_corrected_y][isometric_corrected_x])} on it."
                    
                tiletext= f" This tile is {tile_names.get(map_visible[isometric_corrected_y][isometric_corrected_x])}."

                text_surface = font2.render((f"You clicked (x:{isometric_corrected_x+camera_x-display_tiles_x}, y:{isometric_corrected_y+camera_y-display_tiles_y}).{tiletext}{e_text}{p_text}"), False, (33, 33, 33))
                screen.blit(text_surface,(0,0))

                point1 = map_full[isometric_corrected_y+camera_y-display_tiles_y][isometric_corrected_x+camera_x-display_tiles_x]
                if editor_mode == True:
                    x_to_place = isometric_corrected_x+camera_x-display_tiles_x
                    y_to_place = isometric_corrected_y+camera_y-display_tiles_y
                    if SLC[0]=="tile":
                        map_full[y_to_place][x_to_place] = SLC[1]
                    elif SLC[0]=="entity":
                        entity_full[y_to_place][x_to_place] = SLC[1]
            
            elif editor_mode == True: #if click is inside bottom gui
                    SLC = (gui_buttons_map.get(int(clicked_x/64)))
        
        if (keys[pygame.K_SPACE]):
            la_texta = font2.render(f"{poplist[0].line_of_sight(map_full,entity_full)}", False, (33, 33, 33))
            screen.blit(la_texta,(0,25))

        if (keys[pygame.K_w]):
            poplist[0].move((0,1))
        elif (keys[pygame.K_s]):
            poplist[0].move((0,-1))
        elif (keys[pygame.K_d]):
            poplist[0].move((1,0))
        elif (keys[pygame.K_a]):
            poplist[0].move((-1,0))
        
        timetext= font2.render((f"Day {days}, time: {day_hours}:{mins}"),False,(33,33,33))
        screen.blit(timetext,(0,90+guiwidth[CURRENT_STATE]))
   
        pygame.display.flip()
        FPSCLOCK.tick(60)

    pygame.event.pump()




(eog:21495): Gtk-WARNING **: 22:27:37.792: Could not load a pixbuf from icon theme.
This may indicate that pixbuf loaders or the mime database could not be found.


SystemExit: 

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


        if 0<day_hours<4:
            shade_img = shade5
        elif day_hours<6:
            shade_img = shade4
        elif day_hours<7:
            shade_img = shade3
        elif day_hours<8:
            shade_img = shade1
        elif day_hours<16:
            shade_img = shade0
        elif day_hours<17:
            shade_img = shade2
        elif day_hours<19:
            shade_img = shade3
        elif day_hours<21:
            shade_img = shade4
        elif day_hours<24:
            shade_img = shade5

        for y,row in enumerate(map_visible):
            for x,column in enumerate(row):
                screen.blit(shade_img,(centered_x,centered_y-800))